

## 1.1 Condición de Estabilidad de Courant en FDTD

La **condición de estabilidad de Courant** establece una relación entre el paso espacial (Δz) y el paso temporal (Δt) en los métodos de diferencias finitas en el dominio del tiempo (FDTD).  

## Explicación

- En una simulación FDTD, la onda electromagnética debe propagarse de manera estable a través de la malla numérica.  
- Para que esto ocurra, el avance temporal Δt debe ser lo suficientemente pequeño en comparación con el paso espacial Δz y la velocidad de propagación de la onda en el medio.  

La condición de Courant en 1D se expresa como:

$$
\beta = c \, \frac{\Delta t}{\Delta z} \leq  \frac{1}{2}
$$

donde:
- $c$ es la velocidad de la luz normalizada $c = 1$),
- $\Delta z$ es el tamaño de celda espacial,
- $\Delta t$ es el paso de tiempo.

## Implicaciones

1. **Elección de Δz**  
   - Un paso espacial muy grande $\Delta z$ produce baja resolución espacial, lo que puede introducir errores en la representación de la onda.  
   - Un paso demasiado pequeño aumenta el costo computacional porque se requieren más celdas.

2. **Elección de Δt**  
   - Una vez elegido $\Delta z$, el paso de tiempo $\Delta t$ está limitado por la condición de Courant.  
   - Si $\Delta t$ es demasiado grande, la simulación se vuelve inestable (los campos crecen sin control).  
   - Si $\Delta t$ es muy pequeño, la simulación es estable pero requiere mayor tiempo de cómputo.

## Conclusión

La condición de estabilidad de Courant garantiza que la simulación numérica represente de forma física la propagación de ondas. La elección de $\Delta z$ define la resolución espacial, mientras que $\Delta t$ debe ajustarse de acuerdo con $\Delta z$ para mantener la estabilidad.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros de la malla
Nz = 200           # número de puntos espaciales
dz = 1.0           # paso espacial
z = np.arange(0, Nz*dz, dz)

Ex0 = 0.1*np.sin((2*np.pi*z)/100)  # campo eléctrico inicial
Hy0 = 0.1*np.sin((2*np.pi*z)/100)  # campo magnético inicial

# === Graficar condiciones iniciales ===
plt.figure(figsize=(8,4))
plt.plot(z, Ex0, label="Campo eléctrico Ex (t=0)", color="blue")
plt.plot(z, Hy0, label="Campo magnético Hy (t=0)", color="red", linestyle="--")
plt.xlabel("Posición (z)")
plt.ylabel("Amplitud")
plt.title("Condiciones iniciales del campo electromagnético (t = 0)")
plt.legend()
plt.grid(True)
plt.show()


Con $\Delta t  = 0.5, c = 1, \Delta z = 1$; entonces    $\beta = 0.5  \leq  \frac{1}{2} $ (cumple la condición)

In [ ]:
from IPython.display import Image, display

display(Image(filename="onda1.gif"))
display(Image(filename="onda_emt1.gif"))


Como podemos observar al comparar las condiciones iniciales de los campos magnético y eléctrico con las gráficas generadas en la simulación de la evolución temporal, podemos observar que los perfiles de los ca,pos se mantienen y que ambos conservan la misma fase y la amplitud. Por lo tanto, se puede decir que la simulación es buena.

## 1.2 Estudio de condiciones de frontera y estabilidad numérica

### a) Campos anulándose en el borde, es decir (E = H = 0 en z = 0 y z = 200)

Para hacer esto cambiamos modificamos el código de nuestro archivo fdtd.cpp del proyecto,  código en **C++** que implementa condiciones de frontera CF:

```cpp

#include "fdtd.h"
#include <cmath>
#include <fstream>
#include <iomanip>

// =====================
// Constantes globales
// =====================
constexpr double PI = 3.14159265358979323846;

// =====================
// Constructor
// =====================
Campos::Campos(int xmax_, double c_, double dz_, double lambda_, int decim_)
    : xmax(xmax_), c(c_), dz(dz_), lambda(lambda_), decim(decim_) 
{
    dt = 0.5;  // dt = 0.5 * dz / c;
    beta = c * dt / dz;
    T = lambda / c;
    nsteps = static_cast<int>(20 * T / dt);     // Aquí aumentamos el tiempo de simulación

    Ex.assign(xmax, std::vector<double>(2, 0.0));
    Hy.assign(xmax, std::vector<double>(2, 0.0));
}

// =====================
// Inicialización
// =====================
void Campos::inicializar() {
    for (int k = 0; k < xmax; ++k) {
        double z = k * dz;
        double ini = 0.1 * sin(2.0 * PI * z / lambda);
        Ex[k][0] = ini;
        Hy[k][0] = ini;
    }
}

// =====================
// Simulación
// =====================
void Campos::simular(const std::string &filename) {
    std::ofstream f(filename);
    f << "# t\tk\tEx\tHy\n";
    f << std::fixed << std::setprecision(6);

    for (int n = 0; n < nsteps; ++n) {
        // --- Actualización de Hy ---
        for (int k = 0; k < xmax - 1; ++k) {
            Hy[k][1] = Hy[k][0] + beta * (Ex[k + 1][0] - Ex[k][0]);
        }
        // Frontera derecha 
        Hy[xmax - 1][1] = 0.0;

        // --- Actualización de Ex ---
        for (int k = 1; k < xmax; ++k) {
            Ex[k][1] = Ex[k][0] + beta * (Hy[k][1] - Hy[k - 1][1]);
        }
        // Frontera izquierda 
        Ex[0][1] = 0.0;

        // --- Avance temporal ---
        for (int k = 0; k < xmax; ++k) {
            Ex[k][0] = Ex[k][1];
            Hy[k][0] = Hy[k][1];
        }

        // --- Salida ---
        if (n % decim == 0) {
            double t = n * dt;
            for (int k = 0; k < xmax; ++k) {
                f << t << '\t' << k << '\t' << Ex[k][0] << '\t' << Hy[k][0] << '\n';
            }
        }
    }

    f.close();
}



Con $\Delta t  = 0.5, c = 1, \Delta z = 1$; entonces    $\beta = 0.5  \leq  \frac{1}{2} $ (cumple la condición)

In [ ]:
from IPython.display import Image, display

display(Image(filename="ondaCF.gif"))       # Carga el archivo .gif con las condiciones de frontera
display(Image(filename="onda_emtCF.gif"))

Comparando  las graficas anteriores, con las del las condiciones de frontera períódicas, se observa que:

* Con condiciones periódicas: la onda “reaparece” en el otro extremo y siempre sonserva su perfil

* Con las condiciones de frontera en cero (𝐸 = 𝐻 = 0 en los bordes) :
  - la onda se refleja completamente ⇒ aparecen ondas reflejadas y luego interferencias. En los extremos la onda desaparece.
  - Los pulsos se reflejan totalmente al llegar a las paredes (condiciones de conductor perfecto, PEC).
  - Esto genera ondas reflejadas que interfieren con la onda incidente, creando patrones estacionarios si dejas correr mucho tiempo.

### b) Análisis de estabilidad

#### **I. Variando $\Delta t$ y los demás parámetros fijos**

* Con $\Delta t  = 0.5, c = 1, \Delta z = 1$; entonces    $\beta = 0.5  \leq  \frac{1}{2} $ (cumple la condición)

In [ ]:
from IPython.display import Image, display

display(Image(filename="onda1.gif"))
display(Image(filename="onda_emt1.gif"))


* Con $\Delta t  = 0.1, c = 1, \Delta z = 1$; entonces    $\beta = 0.1  \leq  \frac{1}{2} $ (cumple la condición)

In [ ]:
from IPython.display import Image, display

display(Image(filename="onda2.gif"))
display(Image(filename="onda_emt2.gif"))


* Con $\Delta t  = 2.0, c = 1, \Delta z = 1$; entonces    $\beta = 2.0  >  \frac{1}{2} $ (No cumple la condición de Courant )

In [ ]:
from IPython.display import Image, display

display(Image(filename="onda3.gif"))
display(Image(filename="onda_emt3.gif"))


## **Conclusión**
Como se puede observar en las gráficas anteriores al disminuir $\Delta t$  la simulación es estable pero requiere mayor tiempo de cómputo, es decir, la simulación es más lenta.

Cuando $\Delta t$ es demasiado grande, no se cumple la condición de Courant  y los valores de los campos se disparan al infinito lo que genera que la simulación se vuelva inestable y los campos crecen sin control.  


